In [57]:
import os 
os.getcwd()

'c:\\Work\\Roulet_Prediction\\Roulette_Model_files'

In [58]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

# Step 1: Data Preprocessing
# data = pd.read_excel("NEW DATA SHEET (copy) with features.xlsx")  
data=pd.read_excel("Roulette_data.xlsx")

In [59]:
data.head()

,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2
1,10,D1,C1,EVEN,BLACK,C,G2
2,35,D3,C2,ODD,BLACK,A,G1
3,0,0,0,0,GREEN,A,G1
4,20,D2,C2,EVEN,BLACK,B,G2


In [60]:
data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [61]:
# categorical_columns=['Dozen', 'Column', 'odd/even', 'red / black', 'series', "Group"]   
categorical_columns=['dozen', 'column', 'parity', 'color', 'series', 'group']
# Convert all categorical columns to strings to ensure uniformity
data[categorical_columns] = data[categorical_columns].astype(str)

In [62]:
data.dozen.value_counts()

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64

In [63]:
data.series.value_counts()

series
A    30894
C    21981
B    14527
Name: count, dtype: int64

In [64]:
data.group.value_counts()


group
G1    34591
G2    32811
Name: count, dtype: int64

In [65]:
data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [66]:
#  ['Dozen', 'Column', 'odd/even', 'red / black', 'series','Group']
for i in ['dozen', 'column', 'parity', 'color', 'series', 'group']:
    # print(i)
    print(data[i].value_counts())
    print("\n")

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64


column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64


parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64


color
BLACK    32804
RED      32721
GREEN     1877
Name: count, dtype: int64


series
A    30894
C    21981
B    14527
Name: count, dtype: int64


group
G1    34591
G2    32811
Name: count, dtype: int64




In [67]:
# data.Dozen.replace({"D 1":"D1", "D 2":"D2", "D 3":"D3"}, inplace=True)
# data.Column.replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)
# data["odd/even"].replace({"nan":"0"}, inplace=True)


In [68]:
data.dozen.value_counts()

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64

In [69]:
data.column.value_counts()

column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64

In [70]:
# data.Column.replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)

In [71]:
data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [72]:
data["parity"].value_counts()

parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64

In [73]:
data[data["parity"]=="0"]

,number,dozen,column,parity,color,series,group
3,0,0,0,0,GREEN,A,G1
9,0,0,0,0,GREEN,A,G1
43,0,0,0,0,GREEN,A,G1
182,0,0,0,0,GREEN,A,G1
433,0,0,0,0,GREEN,A,G1
...,...,...,...,...,...,...,...
67165,0,0,0,0,GREEN,A,G1
67262,0,0,0,0,GREEN,A,G1
67331,0,0,0,0,GREEN,A,G1
67347,0,0,0,0,GREEN,A,G1


In [74]:
# data["odd/even"].replace({"nan":0}, inplace=True)

In [75]:
data["series"].value_counts()

series
A    30894
C    21981
B    14527
Name: count, dtype: int64

In [76]:
data["color"].value_counts()

color
BLACK    32804
RED      32721
GREEN     1877
Name: count, dtype: int64

In [77]:
data["group"].value_counts()

group
G1    34591
G2    32811
Name: count, dtype: int64

In [78]:
data["dozen"].value_counts()

dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64

In [79]:
data["column"].value_counts()

column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64

In [80]:
data["parity"].value_counts()

parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64

In [81]:
data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [82]:
# for i in ['Dozen', 'Column', 'odd/even', 'red / black', 'series',
#        'Group']:
#     print(i, data[i].value_counts())

In [83]:
model_data=data.copy()

In [84]:
model_data.head()

,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2
1,10,D1,C1,EVEN,BLACK,C,G2
2,35,D3,C2,ODD,BLACK,A,G1
3,0,0,0,0,GREEN,A,G1
4,20,D2,C2,EVEN,BLACK,B,G2


In [85]:
model_data.columns


Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [86]:
# model_data[model_data["Dozen"]=="0"]

In [87]:
#Remove rows with 0 in Dozen, Column, odd/even, red/black, series, Group
# model_data=model_data[model_data["Dozen"]!="0"]
# model_data[model_data["Column"]=="0"]

In [88]:
for i in ['dozen', 'column', 'parity', 'color', 'series', 'group']:
    print(i, model_data[i].value_counts())

dozen dozen
D1    22014
D2    21870
D3    21641
0      1877
Name: count, dtype: int64
column column
C2    22025
C3    21781
C1    21719
0      1877
Name: count, dtype: int64
parity parity
EVEN    32959
ODD     32566
0        1877
Name: count, dtype: int64
color color
BLACK    32804
RED      32721
GREEN     1877
Name: count, dtype: int64
series series
A    30894
C    21981
B    14527
Name: count, dtype: int64
group group
G1    34591
G2    32811
Name: count, dtype: int64


In [89]:
model_data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [90]:
model_data.head()

,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2
1,10,D1,C1,EVEN,BLACK,C,G2
2,35,D3,C2,ODD,BLACK,A,G1
3,0,0,0,0,GREEN,A,G1
4,20,D2,C2,EVEN,BLACK,B,G2


In [91]:
# for i in ['Dozen', 'Column', 'odd/even', 'red / black', 'series',
#        'Group']:
#     print(i, model_data[i].value_counts())

In [92]:
model_data.dtypes

number     int64
dozen     object
column    object
parity    object
color     object
series    object
group     object
dtype: object

In [93]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Is GPU available: ", tf.test.is_gpu_available())


Num GPUs Available:  0
Is GPU available:  False


In [94]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9962166193300441132
xla_global_id: -1
]


In [95]:
model_data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

### Prediction on Group 

In [96]:
group_data=model_data.copy()

In [97]:
categorical_columns=['dozen', 'column', 'parity', 'color', 'series', 'group']

label_encoder={"dozen": {"0":0,"D1":1, "D2":2,"D3":3},"column": {"0":0,"C1":1, "C2":2,"C3":3}, "parity": {"0":0,"ODD":2, "EVEN":1}, "color": {"RED":2,"GREEN":1, "BLACK":0}, "series": {"A":0,"B":1, "C":2}, "group":{"G1":0, "G2":1}}
# Replace values in each column based on the provided dictionary
for column, mapping in label_encoder.items():
    if column in categorical_columns:
        group_data[column] = group_data[column].map(mapping)

In [98]:
group_data.head()

,number,dozen,column,parity,color,series,group
0,6,1,3,1,0,1,1
1,10,1,1,1,0,2,1
2,35,3,2,2,0,0,0
3,0,0,0,0,1,0,0
4,20,2,2,1,0,1,1


In [99]:
for i in categorical_columns:
    print(i, group_data[i].value_counts())

dozen dozen
1    22014
2    21870
3    21641
0     1877
Name: count, dtype: int64
column column
2    22025
3    21781
1    21719
0     1877
Name: count, dtype: int64
parity parity
1    32959
2    32566
0     1877
Name: count, dtype: int64
color color
0    32804
2    32721
1     1877
Name: count, dtype: int64
series series
0    30894
2    21981
1    14527
Name: count, dtype: int64
group group
0    34591
1    32811
Name: count, dtype: int64


In [100]:
group_data=group_data.drop(["number"], axis=1)

In [101]:
group_data.head()

,dozen,column,parity,color,series,group
0,1,3,1,0,1,1
1,1,1,1,0,2,1
2,3,2,2,0,0,0
3,0,0,0,1,0,0
4,2,2,1,0,1,1


In [102]:
data.columns

Index(['number', 'dozen', 'column', 'parity', 'color', 'series', 'group'], dtype='object')

In [103]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

def preprocess_data(data):
    # Check for columns with non-numeric data
    print("Checking data types before preprocessing:")
    print(data.dtypes)

    # Normalize 'Dozen', "series" and 'Column' values (if they are now numeric)
    scaler = StandardScaler()
    data[['dozen', 'column', 'parity', 'color', 'series']] = scaler.fit_transform(data[['dozen', 'column', 'parity', 'color', 'series']])

    # Fill missing values if necessary
    data = data.fillna(0)
    
    return data

# data = preprocess_data(data)



In [104]:
data=preprocess_data(group_data)

Checking data types before preprocessing:
dozen     int64
column    int64
parity    int64
color     int64
series    int64
group     int64
dtype: object


In [105]:
data.head() 

,dozen,column,parity,color,series,group
0,-1.080172,1.214899,-0.826206,-1.012974,0.150993,1
1,-1.080172,-1.088716,-0.826206,-1.012974,1.292836,1
2,1.221077,0.063091,0.988384,-1.012974,-0.990850,0
3,-2.230796,-2.240524,-2.640796,0.001249,-0.990850,0
4,0.070452,0.063091,-0.826206,-1.012974,0.150993,1


In [106]:
# # Step 2: Prepare features (X) and target (y)
# # We will predict 'series' for the next round

# X = []
# y = []

# sequence_length = 5  # Number of previous rounds to consider as input
# for i in range(len(data) - sequence_length):
#     # print("-------------------")
#     # print(i+sequence_length)
#     # print(data.iloc[i:i+sequence_length].drop(columns=['Group']).values)
#     X.append(data.iloc[i:i+sequence_length].drop(columns=['Group']).values)# Features (all columns except 'series')
    
#     # print(data.iloc[i+sequence_length]['Group'])
#     # print("________________")
#     y.append(data.iloc[i+sequence_length]['Group'])  # Target (next round's 'series')

# X = np.array(X)
# y = np.array(y)

# # Reshape X for LSTM [samples, time steps, features]
# X = X.reshape(X.shape[0], X.shape[1], X.shape[2])




In [107]:
labels = data['group'].values

In [110]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [111]:
data.head()

,dozen,column,parity,color,series,group
0,-1.080172,1.214899,-0.826206,-1.012974,0.150993,1
1,-1.080172,-1.088716,-0.826206,-1.012974,1.292836,1
2,1.221077,0.063091,0.988384,-1.012974,-0.990850,0
3,-2.230796,-2.240524,-2.640796,0.001249,-0.990850,0
4,0.070452,0.063091,-0.826206,-1.012974,0.150993,1


In [112]:
data[:50]

,dozen,column,parity,color,series,group
0,-1.080172,1.214899,-0.826206,-1.012974,0.150993,1
1,-1.080172,-1.088716,-0.826206,-1.012974,1.292836,1
2,1.221077,0.063091,0.988384,-1.012974,-0.990850,0
3,-2.230796,-2.240524,-2.640796,0.001249,-0.990850,0
4,0.070452,0.063091,-0.826206,-1.012974,0.150993,1
5,-1.080172,1.214899,-0.826206,1.015471,-0.990850,0
6,1.221077,1.214899,-0.826206,1.015471,1.292836,1
7,-1.080172,-1.088716,-0.826206,-1.012974,-0.990850,0
8,-1.080172,-1.088716,0.988384,1.015471,-0.990850,0
9,-2.230796,-2.240524,-2.640796,0.001249,-0.990850,0


In [113]:


# Step 2: Prepare features (X) and target (y)
# We will predict 'series' for the next round

X = []
y = []
time_steps = 10
num_features=list(np.unique(labels))
# sequence_length = 5  # Number of previous rounds to consider as input
for i in range(len(data) - time_steps):
    # print("-------------------")
    # print(data.iloc[i:i+sequence_length].drop(columns=['Group']).values)
    X.append(data.iloc[i:i+time_steps].drop(columns=['group']).values)# Features (all columns except 'series')
    y.append(data.iloc[i+time_steps]['group'])  # Target (next round's 'series')

X = np.array(X)
y = np.array(y)

# Reshape X for LSTM [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])
num_features = X.shape[-1]


In [114]:
data

,dozen,column,parity,color,series,group
0,-1.080172,1.214899,-0.826206,-1.012974,0.150993,1
1,-1.080172,-1.088716,-0.826206,-1.012974,1.292836,1
2,1.221077,0.063091,0.988384,-1.012974,-0.990850,0
3,-2.230796,-2.240524,-2.640796,0.001249,-0.990850,0
4,0.070452,0.063091,-0.826206,-1.012974,0.150993,1
...,...,...,...,...,...,...
67397,-1.080172,1.214899,-0.826206,1.015471,-0.990850,0
67398,-1.080172,1.214899,-0.826206,-1.012974,0.150993,1
67399,-1.080172,0.063091,0.988384,1.015471,1.292836,1
67400,-1.080172,0.063091,0.988384,1.015471,1.292836,1


In [115]:

# # Step 2: Prepare features (X) and target (y)
# # We will predict 'series' for the next round

# X = []
# y = []
# time_steps = 50
# num_features=list(np.unique(labels))
# # sequence_length = 5  # Number of previous rounds to consider as input
# for i in range(len(data) - time_steps):
#     # print("-------------------")
#     # print(data.iloc[i:i+sequence_length].drop(columns=['Group']).values)
#     X.append(data.iloc[i:i+time_steps].drop(columns=['Group']).values)# Features (all columns except 'series')
#     y.append(data.iloc[i+time_steps]['Group'])  # Target (next round's 'series')

# X = np.array(X)
# y = np.array(y)

# # Reshape X for LSTM [samples, time steps, features]
# X = X.reshape(X.shape[0], X.shape[1], X.shape[2])
# num_features = X.shape[-1]

In [116]:
X[50]

array([[-1.08017199,  0.0630912 ,  0.98838375, -1.01297351,  1.29283608],
       [ 1.22107661, -1.08871625, -0.82620622, -1.01297351, -0.99084963],
       [ 0.07045231,  0.0630912 , -0.82620622, -1.01297351,  0.15099323],
       [ 1.22107661, -1.08871625, -0.82620622,  1.01547137,  0.15099323],
       [ 1.22107661,  0.0630912 ,  0.98838375, -1.01297351, -0.99084963],
       [ 1.22107661, -1.08871625,  0.98838375,  1.01547137, -0.99084963],
       [ 1.22107661, -1.08871625,  0.98838375, -1.01297351,  0.15099323],
       [-1.08017199,  0.0630912 , -0.82620622, -1.01297351, -0.99084963],
       [ 1.22107661,  0.0630912 ,  0.98838375, -1.01297351, -0.99084963],
       [ 0.07045231,  0.0630912 , -0.82620622,  1.01547137,  0.15099323]])

In [117]:
y[50]

np.float64(1.0)

In [118]:
data["group"].value_counts()

group
0    34591
1    32811
Name: count, dtype: int64

In [119]:
num_features

5

In [120]:
# Train-Test Split (No separate validation set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)


In [121]:
print(X_train.shape)  # Expected: (number_of_samples, time_steps, num_features)


(66718, 10, 5)


In [122]:
X_train=X
y_train=y

In [123]:
# model = Sequential([
#     LSTM(64, activation='tanh', return_sequences=True, input_shape=(time_steps, num_features)),
#     Dropout(0.2),
#     LSTM(32, activation='tanh'),
#     Dropout(0.2),
#     Dense(32, activation='tanh'),
#     Dense(1, activation='sigmoid')  # Binary classification
# ])

# # Compile the Model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # ================================
# # Step 3: Train the Model
# # ================================
# # Callbacks for early stopping and saving the best model
# early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model2.keras', monitor='loss', save_best_only=True)

# # Train the model
# history = model.fit(
#     X, y,
#     epochs=50,
#     batch_size=32,
#     callbacks=[early_stopping, model_checkpoint]
# )


In [124]:
X_train.shape


(67392, 10, 5)

In [125]:
num_classes = data["group"].nunique()  # Number of classes
time_steps = X_train.shape[1]
num_features = X_train.shape[2]

# Build the LSTM Model
model = Sequential([
    LSTM(64, activation='tanh', return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.3),
    LSTM(32, activation='tanh'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  # Use 4 units for 4 classes (0, 1, 2, 3)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ================================
# Step 3: Train the Model
# ================================
# Callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_odd_even.keras', monitor='loss', save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy[1]*100:.2f}%")


Epoch 1/50


c:\Work\Roulet_Prediction\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1053/1053 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5107 - loss: 0.6931
Epoch 2/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5143 - loss: 0.6929
Epoch 3/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5170 - loss: 0.6925
Epoch 4/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5130 - loss: 0.6929
Epoch 5/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5131 - loss: 0.6928
Epoch 6/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5152 - loss: 0.6928
Epoch 7/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5141 - loss: 0.6928
Epoch 8/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5141 - loss: 0.6928
Epoch 9/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5165 - loss: 0.6926
Epoch 10/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5117 - loss: 0.6929
Epoch 11/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5124 - loss: 0.6928
Epoch 12/50
1053/1053 ━━━━━━━━━━━━━━━━━━━

In [126]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# # Build the LSTM Model
# model = Sequential([
#     Bidirectional(LSTM(64, activation='tanh', return_sequences=True, 
#                        recurrent_dropout=0.2, kernel_regularizer=l2(0.01), 
#                        input_shape=(time_steps, num_features))),
#     Dropout(0.3),
#     Bidirectional(LSTM(32, activation='tanh', recurrent_dropout=0.2, kernel_regularizer=l2(0.01))),
#     Dropout(0.3),
#     Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
#     Dropout(0.3),
#     Dense(num_classes, activation='softmax')
# ])

# # Compile the model with gradient clipping
# optimizer = Adam(learning_rate=0.001, clipvalue=1.0)
# model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Callbacks for early stopping and saving the best model
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model_odd_even.keras', monitor='val_loss', save_best_only=True)

# # Train the model
# history = model.fit(
#     X_train, y_train,
#     epochs=50,
#     batch_size=64,
#     validation_split=0.2,
#     callbacks=[early_stopping, model_checkpoint]
# )

# # Evaluate the model
# accuracy = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {accuracy[1]*100:.2f}%")


In [127]:
# # ================================
# num_classes = data["dozen"].nunique()  # Number of classes
# time_steps = 50
# num_features = X_train.shape[2]

# # Build the LSTM Model
# model = Sequential([
#     LSTM(64, activation='tanh', return_sequences=True, input_shape=(time_steps, num_features)),
#     Dropout(0.2),
#     LSTM(32, activation='tanh'),
#     Dropout(0.2),
#     Dense(32, activation='relu'),
#     Dense(num_classes, activation='softmax')  # Use 4 units for 4 classes (0, 1, 2, 3)
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # ================================
# # Step 3: Train the Model
# # ================================
# # Callbacks for early stopping and saving the best model
# early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('best_model_odd_even.keras', monitor='loss', save_best_only=True)

# # Train the model
# history = model.fit(
#     X_train, y_train,
#     epochs=50,
#     batch_size=64,
#     callbacks=[early_stopping, model_checkpoint]
# )

# # Evaluate the model
# accuracy = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {accuracy[1]*100:.2f}%")


In [128]:
# ================================
# Step 4: Evaluate the Model
# ================================
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict on the test set
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

22/22 - 0s - 3ms/step - accuracy: 0.5964 - loss: 0.6250
Test Loss: 0.6250
Test Accuracy: 0.5964
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [129]:
X_test.shape
X_test[:1].shape

(1, 10, 5)

In [130]:
X_test[:1]

array([[[-1.08017199,  0.0630912 ,  0.98838375,  1.01547137,
          1.29283608],
        [-1.08017199,  0.0630912 , -0.82620622, -1.01297351,
          1.29283608],
        [-1.08017199, -1.08871625, -0.82620622, -1.01297351,
          1.29283608],
        [-1.08017199,  0.0630912 ,  0.98838375,  1.01547137,
          1.29283608],
        [-1.08017199, -1.08871625, -0.82620622, -1.01297351,
          1.29283608],
        [-1.08017199,  1.21489865,  0.98838375,  1.01547137,
          0.15099323],
        [ 1.22107661,  0.0630912 ,  0.98838375, -1.01297351,
         -0.99084963],
        [ 1.22107661, -1.08871625, -0.82620622, -1.01297351,
         -0.99084963],
        [-1.08017199,  0.0630912 , -0.82620622, -1.01297351,
         -0.99084963],
        [ 1.22107661,  0.0630912 ,  0.98838375, -1.01297351,
         -0.99084963]]])

In [131]:
y_test

array([0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 1.

In [132]:
#Predict on the test set
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
print(y_pred)

# # Save the model
# model.save("Group_lstm_model.h5")

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[0 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1
 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0
 0 0 1 1 1 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0
 1 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1
 1 0 0 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0
 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 1
 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0
 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 1
 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1
 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0
 0 1 1 0 1 0 1 0 1 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 0
 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0
 0 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1

In [134]:
# Save the model
model.save("Group_lstm_model_10_1.h5")

In [84]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_train)
y_pred=np.argmax(y_pred, axis=1)
print(y_pred)
print(accuracy_score(y_train, y_pred))

2106/2106 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
[0 0 0 ... 0 0 0]
0.5131914767331434


In [160]:
test_accuracy = np.sum(y_pred == y_train) / len(y_train)
test_accuracy

np.float64(0.5127984321178287)

### Inference Script


In [65]:
# data2=pd.read_excel("NEW DATA SHEET (copy) with features.xlsx") 
data2=pd.read_excel("Roulette_data.xlsx")

In [66]:
from tensorflow.keras.models import load_model

# Load the trained model
model_path = "Group_lstm_model.h5"
model = load_model(model_path)
print("Model loaded successfully.")

Model loaded successfully.


In [67]:
model.predict(X_test[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


array([[0.2859168]], dtype=float32)

In [68]:
data2[:1]


,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2


In [69]:
import warnings
warnings.filterwarnings("ignore")   

In [75]:
X.shape

(67352, 50, 5)

In [76]:
import numpy as np
from tensorflow.keras.models import load_model

# ================================
# Step 1: Load the Trained Model
# ================================
def load_trained_model(model_path):
    """Load the trained model from a file."""
    model = load_model(model_path)
    print("Model loaded successfully.")
    return model

# ================================
# Step 2: Preprocess Input Data
# ================================
def preprocess_input(input_data):
    """Preprocess input data to match the model's expected shape."""
    # Ensure input_data is a NumPy array

    categorical_columns=['dozen', 'column', 'parity', 'color', 'series', 'group']

    # Convert all categorical columns to strings to ensure uniformity
    input_data[categorical_columns] = input_data[categorical_columns].astype(str)
    # #Preprocess categorical columns
    # input_data = input_data.drop(["Number"], axis=1)
    # input_data["Dozen"].replace({"D 1":"D1", "D 2":"D2", "D 3":"D3"}, inplace=True)
    # input_data["Column"].replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)
    # input_data["odd/even"].replace({"nan":"0"}, inplace=True)

    label_encoders = {}
    for column in categorical_columns:
        le = LabelEncoder()
        input_data[column] = le.fit_transform(input_data[column])
        label_encoders[column] = le

    #Scale the data
    scaler = StandardScaler()
    input_data[['dozen', 'column', 'parity', 'color', 'series']] = scaler.fit_transform(input_data[['dozen', 'column', 'parity', 'color', 'series']])
    # Fill missing values if necessary
    input_data = input_data.fillna(0)
    ## Converting Data Time Series
    # X = []
    # y=[]
    time_steps = 50
    input_data=input_data.drop(["number"], axis=1)
    X=input_data.drop(columns=['group']).values
    X= np.array(X)
    # y = np.array(y)
    # print(X.shape)
    X = X.reshape(1, time_steps, X.shape[1])
    # input_data = np.array(input_data)

    return X 

# ================================
# Step 3: Perform Inference
# ================================
def predict_class(model, X):
    """Predict class probabilities and labels for the input data."""
    # Get probabilities
    probabilities = model.predict(X)

    # Convert probabilities to binary labels (0 or 1)
    predicted_labels = (probabilities > 0.5).astype(int)
    # print(predicted_labels)
    decoded_labels = label_encoders["group"].inverse_transform(predicted_labels)
    
    return probabilities, predicted_labels ,decoded_labels
# ================================
# Step 4: Main Inference Script
# ================================
if __name__ == "__main__":
    # Path to the trained model
    model_path = "best_model2.keras"

    # Load the model
    model = load_trained_model(model_path)

    # Example input data (replace with actual test input)
    # Example assumes time_steps=10, num_features=5
    
    sample_input =data2[:50]  # Replace with your actual data

    # Preprocess input data
    preprocessed_input = preprocess_input(sample_input)

    # Perform inference
    probabilities, predicted_labels,decoded_labels = predict_class(model, preprocessed_input)

    # Output the results
    print("Predicted Probabilities:", probabilities[-1])
    print("Predicted Labels:", predicted_labels[-1])
    print("Decoded Labels:", decoded_labels[-1])


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
Predicted Probabilities: [0.2232356]
Predicted Labels: [0]
Decoded Labels: G1


In [74]:
data2

,number,dozen,column,parity,color,series,group
0,6,D1,C3,EVEN,BLACK,B,G2
1,10,D1,C1,EVEN,BLACK,C,G2
2,35,D3,C2,ODD,BLACK,A,G1
3,0,0,0,0,GREEN,A,G1
4,20,D2,C2,EVEN,BLACK,B,G2
...,...,...,...,...,...,...,...
67397,12,D1,C3,EVEN,RED,A,G1
67398,6,D1,C3,EVEN,BLACK,B,G2
67399,5,D1,C2,ODD,RED,C,G2
67400,5,D1,C2,ODD,RED,C,G2


In [326]:
data2[:1]

,Number,Dozen,Column,odd/even,red / black,series,Group
0,6,D 1,C 3,EVEN,BLACK,B,G2


In [330]:
model.predict(preprocessed_input[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step


array([[0.6777578]], dtype=float32)

In [388]:
# import numpy as np
# import pandas as pd
# from tensorflow.keras.models import load_model
# from sklearn.preprocessing import StandardScaler

# # ================================
# # Step 1: Load the Trained Model
# # ================================
# def load_trained_model(model_path):
#     """Load the trained model from a file."""
#     model = load_model(model_path)
#     print("Model loaded successfully.")
#     return model

# # ================================
# # Step 2: Preprocess Input Data
# # ================================
# def preprocess_input(input_data, label_encoders, time_steps, num_features):
#     """Preprocess input data to match the model's expected shape."""
#     # Ensure input_data is a DataFrame
#     categorical_columns = ['Dozen', 'Column', 'odd/even', 'red / black', 'series', 'Group']

#     # Convert all categorical columns to strings to ensure uniformity
#     input_data[categorical_columns] = input_data[categorical_columns].astype(str)

#     # Convert all categorical columns to strings to ensure uniformity
#     input_data[categorical_columns] = input_data[categorical_columns].astype(str)
#     #Preprocess categorical columns
#     input_data = input_data.drop(["Number"], axis=1)
#     input_data["Dozen"].replace({"D 1":"D1", "D 2":"D2", "D 3":"D3"}, inplace=True)
#     input_data["Column"].replace({"C 1":"C1", "C 2":"C2", "C 3":"C3"}, inplace=True)
#     input_data["odd/even"].replace({"nan":"0"}, inplace=True)
    
#     # Encoding categorical columns
#     for column in categorical_columns:
#         input_data[column] = label_encoders[column].transform(input_data[column])
    
#     # Normalize 'Dozen', "series" and 'Column' values
#     scaler = StandardScaler()
#     input_data[['Dozen', 'Column', 'odd/even', 'red / black', 'series']] = scaler.fit_transform(input_data[['Dozen', 'Column', 'odd/even', 'red / black', 'series']])

#     # Fill missing values if necessary
#     input_data = input_data.fillna(0)

#     # Converting Data Time Series
#     X = input_data.drop(columns=['Group'])
#     y = input_data['Group']

#     if len(X) < time_steps:
#         padding = pd.DataFrame(np.zeros((time_steps - len(X), num_features)))
#         X = pd.concat([padding, X], ignore_index=True)
#     X = np.array(X.values)
#     y = np.array(y.values)
#     X.reshape(1, time_steps, num_features)

#     return X, y

# # ================================
# # Step 3: Perform Inference
# # ================================
# def predict_class(model, X):
#     """Predict class probabilities and labels for the input data."""
#     # Get probabilities
#     probabilities = model.predict(X)

#     # Convert probabilities to binary labels (0 or 1)
#     predicted_labels = (probabilities > 0.5).astype(int)
#     return probabilities, predicted_labels

# # ================================
# # Step 4: Main Inference Script
# # ================================
# if __name__ == "__main__":
#     # Path to the trained model
#     model_path = "lstm_model.h5"

#     # Load the model
#     model = load_trained_model(model_path)

#     # Example input data (replace with actual test input)
#     sample_input = pd.read_excel("NEW DATA SHEET (copy) with features.xlsx")[:1]  # Replace with your actual data
#     time_steps = 50
#     num_features = 5
#     # Preprocess input data
#     X, y = preprocess_input(sample_input, label_encoders, time_steps, num_features)

#     # Perform inference
#     probabilities, predicted_labels = predict_class(model, X)

#     # Output the results
#     print("Predicted Probabilities:", probabilities)
#     print("Predicted Labels:", predicted_labels)